In [5]:
import torch
import numpy as np
from transformers import BertTokenizer, BertForTokenClassification, BertConfig
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize  
from transformers import pipeline

In [6]:
MODELS_DIR = [
    r"pucpr/clinicalnerpt-diagnostic",
    r"pucpr/clinicalnerpt-disease",           
    r"pucpr/clinicalnerpt-sign",
    r"pucpr/clinicalnerpt-disorder",             
    r"pucpr/clinicalnerpt-finding"   
]

In [221]:
dataset = pd.read_csv('sample_laudos.csv')
dataset

,docid,modalidade,tipo_exame,laudo_completo
0,189988,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,o estudo tomográfico computadorizado do tórax ...
1,376443,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,realizados cortes tomográficos axiais com técn...
2,1843933,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,tecnica: \n o estudo tomográfico computadoriza...
3,454421,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,o estudo tomográfico computadorizado do tórax ...
4,61595,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,realizada tomografia computadorizada do tórax ...
...,...,...,...,...
4995,948854,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,tc do tórax \n técnica: obtidas imagens axiais...
4996,1668375,CR,RADIOGRAFIA DO TÓRAX,partes moles sem particularidades. \n sinais d...
4997,527535,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,o estudo tomográfico computadorizado do tórax ...
4998,131674,CT,TOMOGRAFIA COMPUTADORIZADA DO TÓRAX,infiltrado intersticio-alveolar com broncogram...


In [210]:
def sent_to_words(sent):
    sent = re.sub("\s+", " ", sent)  # remove newline chars
    sent = re.sub("\'", "", sent)  # remove single quotes
    sent = re.sub("  ", " ", sent)  # remove single quotes
    sent = re.sub("\(", "", sent)  # remove single quotes
    sent = re.sub("\)", "", sent)  # remove single quotes
    #sent = simple_preprocess(str(sent), deacc=True) 
    return sent

In [237]:
test_sentence = sent_to_words(dataset['laudo_completo'][67])
test_sentence

'realizados cortes axiais sem a injeção de meio de contraste iodado endovenoso: estudo comparativo com exame anterior do dia 04 04 2014 mostra: permanecem extensas áreas de consolidações pulmonares com broncograma aéreos de permeio comprometendo os lobos superior médio e inferior do pulmão direito com atelectasias adjacentes provavelmente relacionada a processo inflamatório infeccioso na dependência da correlação clínico laboratorial. minimo derrame pleural bilateral menor em relação a estudo anterior. opacidades atelectásicas nos lobos inferiores. estruturas vasculares mediastinais de calibre e trajeto preservados. coração com dimensões preservadas. ausência de linfonodomegalias mediastinais. traquéia e brônquios principais pérvios e com calibre preservado. ausência de derrame pericardico significativo.'

In [272]:
all_entities = []
entities_index = []

for model_name in MODELS_DIR:
    
    model = BertForTokenClassification.from_pretrained(model_name)
    
    tokenizer = BertTokenizer.from_pretrained("pucpr/biobertpt-all", do_lower_case=True)
    
    config = BertConfig.from_pretrained(model_name)
    dic_label=config.id2label

    tokenized_sentence = tokenizer.encode(test_sentence)

    input_ids = torch.tensor([tokenized_sentence])#.cuda()

    with torch.no_grad():
        output = model(input_ids)
    
    label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

    idx = 1
    enc = [tokenizer.encode(x, add_special_tokens=False) for x in test_sentence.split()]
    desired_output = []

    for token in enc:
        tokenoutput = []
        for ids in token:
            tokenoutput.append(idx)
            idx +=1
        desired_output.append(tokenoutput)

    word_id_list = []
    labels = []
    teste = []
    
    for word_id in desired_output:
        idx = word_id[-1]
        labels.append(label_indices[0][idx])
        entities = [dic_label.get(e) for e in labels]
        
    for v in entities:
        if not 'O' in v:
            entities_index.append(entities.index(v))
    
    all_entities.append(entities)

In [264]:
for p in all_entities:
    

[['O', 'B-DiagnosticProcedure', 'I-DiagnosticProcedure', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [274]:
print(all_entities)

[['O', 'B-DiagnosticProcedure', 'I-DiagnosticProcedure', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [275]:
entities_index

[1, 2, 50, 51, 51, 50, 78, 79, 79, 78, 79, 79, 78]